In [ ]:
import torch
import torchvision
import csv
import numpy as np
import torch.nn.functional as F
from torch import Tensor
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
import os
import pydicom
from PIL import Image
import random
import nibabel as nib
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
import sklearn.metrics
import matplotlib
import natsort
device = "cuda" if torch.cuda.is_available() else "cpu"
device_name = 'F' #E
if device_name == 'F':
    torch.cuda.set_device(0)
else:
    torch.cuda.set_device(1)

In [ ]:
epochs = 5
batch_size=5

class Data_load:
    def __init__(self):
        self.len_=0
        self.x_data_name=[]
        self.x_data_test_name=[]
        self.normal_size=0
        self.mci_size=0
        self.ad_size=0
        self.test_len=0
        list_disease = ['Normal', 'MCI', 'AD']
        for i in range(1,6,1):
            for label_target, j in enumerate(list_disease):
                part_data_name=[]
                path = device_name+":/k-fold validation/"+str(i)+"/"+str(j)+"/"
                dir_list = natsort.natsorted(os.listdir(path))
                len_dir=len(dir_list)
                for k in range(len_dir):
                    part_data_name.append([path+dir_list[k], label_target])
                
                self.x_data_name.append(part_data_name)
                
        self.final_data_name=[]

        for i in range(len(self.x_data_name)):
            for j in range(len(self.x_data_name[i])):
                self.final_data_name.append(self.x_data_name[i][j])

        random.shuffle(self.final_data_name)
        
    def out(self):
        return self.final_data_name
    
    def random(self):
        random.shuffle(self.final_data_name)
        return self.final_data_name

def training(data, batch_size):
    batch_total = len(data)//batch_size
    batch_na = len(data)%batch_size
    for i in range(batch_total+1):
        x_data=[]
        y_data=[]
        extra = []
        extra_final = []
        if i == batch_total:
            if batch_na == 0:
                break
            else:
                temp_data = data[i*batch_size:] 
        else:
            temp_data = data[i*batch_size:(i+1)*batch_size]
        for j in range(len(temp_data)):
            img = nib.load(temp_data[j][0]+'/1/final/1_t1_final.mnc').get_fdata() 
            img = img/img.max() 
            x_data.append(img)      

        x_final = torch.from_numpy(np.array(x_data)[:,np.newaxis,:,:,:]).type(torch.FloatTensor).to("cuda")
        y_final = torch.from_numpy(np.array(x_data)[:,np.newaxis,:,:,:]).type(torch.FloatTensor).to("cuda")

        yield (x_final, y_final)

In [ ]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        """
        self.cnn_layer1 = nn.Sequential(
                        nn.Conv3d(1, 64, 5, stride=2),
                        nn.LeakyReLU(),
                        nn.MaxPool3d(2)
                       )

        self.cnn_layer2 = nn.Sequential(
                                nn.Conv3d(64, 128, 3, stride=2),
                                nn.LeakyReLU(),
                            )

        # Decoder
        self.tran_cnn_layer1 = nn.Sequential(
                        nn.ConvTranspose3d(128, 256, 3, stride = 2),
                        nn.LeakyReLU(),
                        nn.Upsample(size=(44, 53, 44)))

        self.tran_cnn_layer2 = nn.Sequential(
                        nn.ConvTranspose3d(256, 1, 5, stride = 2),
                        nn.Upsample(size=(181, 217, 181)),
                        nn.Sigmoid()
                       )
        
        """
        self.layer1 = nn.Conv3d(1, 64, 5, stride=2)
        self.layer2 = nn.MaxPool3d(2)
        self.layer3 = nn.Conv3d(64, 128, 3, stride=2)
        self.layer4 = nn.ConvTranspose3d(128, 256, 3, stride = 2)
        self.layer5 = nn.ConvTranspose3d(256, 1, 5, stride = 2)
        self.layer6 = nn.Upsample(size=(181, 217, 181))
        
    def forward(self, x):
        """
        #print(x.shape)
        output = self.cnn_layer1(x)
        #print(output.shape)
        self.output = self.cnn_layer2(output)
        #print(self.output.shape)
        output = self.tran_cnn_layer1(self.output)
        #print(output.shape)
        output = self.tran_cnn_layer2(output)
        #print(output.shape)
        """
        x = self.layer1(x)
        #print(x.shape)
        x = self.layer2(x)
        #print(x.shape)
        self.output = self.layer3(x)
        #print(x.shape)
        x = self.layer4(self.output)
        #print(x.shape)
        x = self.layer5(x)
        #print(x.shape)
        output = self.layer6(x)
        #print(x.shape)
        #print(x.shape)
        
        return output, self.output

In [ ]:
model = ConvAutoEncoder().to("cuda")
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
data = Data_load()
for t in range(epochs):
    loss = 0
    X_training = data.random()
    print(len(X_training),t)
    model.train()
    for batch, (X, y) in enumerate(training(X_training,batch_size)):
        pred, _ = model(X)
        batch_loss_result = loss_fn(pred, y)
        optimizer.zero_grad()
        batch_loss_result.backward()
        optimizer.step()
        loss+=batch_loss_result
    print("final: ", loss/(batch+1))
#torch.save(model,"3D_autoencoder.pt")


In [ ]:
import torch
import torchvision
import csv
import numpy as np
import torch.nn.functional as F
from torch import Tensor
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
import os
import pydicom
from PIL import Image
import random
import nibabel as nib
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
import sklearn.metrics
import matplotlib
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

device = "cuda" if torch.cuda.is_available() else "cpu"
device_name = 'E' #E
torch.cuda.set_device(0)

class Data_load:
    def __init__(self,model):
        with torch.no_grad():
            self.len_=0
            self.x_data_name=[]
            self.array_x=[]
            self.array_y=[]
            self.model = model
            list_disease = ['Normal', 'MCI', 'AD']
            for i in range(1,6,1):
                for label_target, j in enumerate(list_disease):
                    part_data_name=[]
                    path = device_name+":/k-fold validation/"+str(i)+"/"+str(j)+"/"
                    dir_list = natsort.natsorted(os.listdir(path))
                    len_dir=len(dir_list)
                    for k in range(len_dir):
                        part_data_name.append([path+dir_list[k], label_target])

                    self.x_data_name.append(part_data_name)

            self.final_data_name=[]

            for i in range(len(self.x_data_name)):
                for j in range(len(self.x_data_name[i])):
                    self.final_data_name.append(self.x_data_name[i][j])

            for i in range(len(self.final_data_name)):
                print(i)
                img = nib.load(self.final_data_name[i][0]+'/1/final/1_t1_final.mnc').get_fdata() 
                img = img/img.max() 
                img = torch.from_numpy(img[np.newaxis,np.newaxis,:,:,:]).type(torch.FloatTensor).to("cuda")
                #print(self.model(img)[1].to("cpu").numpy().shape)
                self.array_x.append(self.model(img)[1].to("cpu").numpy())
                self.array_y.append(self.final_data_name[i][1])

            self.array_x = np.array(self.array_x).reshape((len(self.array_x),-1)).astype(np.float64)
            self.array_y = np.array(self.array_y)

data = Data_load(model)

In [ ]:
#print(len(data.array_x))
print(data.array_x.shape, data.array_y.shape)
print(data.array_x.dtype)
#print(data.array_x,data.array_y)

In [ ]:

model_tsne = TSNE(n_components=2)
#print(i)
result = model_tsne.fit_transform(data.array_x)
#print('done')
Normal_list=list()
MCI_list=list()
AD_list=list()
for i in range(result.shape[0]):
    if data.array_y[i]==0:
        Normal_list.append(result[i])
    elif data.array_y[i]==1:
        MCI_list.append(result[i])
    else:
        AD_list.append(result[i])

Normal_list = np.array(Normal_list)
MCI_list = np.array(MCI_list)
AD_list = np.array(AD_list)
#print(Normal_list.shape)
#print(MCI_list.shape)
#print(AD_list.shape)
fig = plt.figure(figsize=(8, 8))

plt.scatter(Normal_list[:,0], Normal_list[:, 1], marker='o', c='r')
plt.scatter(MCI_list[:,0], MCI_list[:, 1], marker='s', c='g')
plt.scatter(AD_list[:,0], AD_list[:, 1], marker='^', c='b')

plt.legend(('Normal','MCI','Dementia'))
#plt.title("Thickness",fontdict = {'fontsize' : 30})

plt.show()

In [ ]:
import torch
import torchvision
import csv
import numpy as np
import torch.nn.functional as F
from torch import Tensor
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
import os
import pydicom
from PIL import Image
import random
import nibabel as nib
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
import sklearn.metrics
import matplotlib
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

device = "cuda" if torch.cuda.is_available() else "cpu"
device_name = 'E' #E
torch.cuda.set_device(0)


class Data_load:
    def __init__(self,model):
        with torch.no_grad():
            self.len_=0
            self.x_data_name=[]
            self.array_x=[]
            self.array_y=[]
            self.model = model
            list_disease = ['Normal', 'MCI', 'AD']
            for i in range(1,6,1):
                for label_target, j in enumerate(list_disease):
                    part_data_name=[]
                    path = device_name+":/k-fold validation/"+str(i)+"/"+str(j)+"/"
                    dir_list = natsort.natsorted(os.listdir(path))
                    len_dir=len(dir_list)
                    for k in range(len_dir):
                        part_data_name.append([path+dir_list[k], label_target])

                    self.x_data_name.append(part_data_name)

            self.final_data_name=[]

            for i in range(len(self.x_data_name)):
                for j in range(len(self.x_data_name[i])):
                    self.final_data_name.append(self.x_data_name[i][j])
            #print(self.final_data_name)        
            for i in range(len(self.final_data_name)):
                print(i)
                file = open(self.final_data_name[i][0]+"/1/thickness/1_native_rms_rsl_tlink_30mm_left.txt", "r")
                lines = file.readlines()
                lines = list(map(float, lines))

                file = open(self.final_data_name[i][0]+"/1/thickness/1_native_rms_rsl_tlink_30mm_right.txt", "r")
                lines_right = file.readlines()
                lines_right = list(map(float, lines_right))

                self.array_x.append(lines +lines_right)
                self.array_y.append(self.final_data_name[i][1])


            self.array_x = np.array(self.array_x).reshape((len(self.array_x),-1)).astype(np.float64)
            self.array_y = np.array(self.array_y)

data1 = Data_load(model)

In [ ]:
#print(len(data.array_x))
print(data1.array_x.shape, data1.array_y.shape)
print(data1.array_x.dtype)
#print(data.array_x,data.array_y)

In [ ]:

model_tsne = TSNE(n_components=2)
#print(i)
result = model_tsne.fit_transform(data1.array_x)
#print('done')
Normal_list=list()
MCI_list=list()
AD_list=list()
for i in range(result.shape[0]):
    if data1.array_y[i]==0:
        Normal_list.append(result[i])
    elif data1.array_y[i]==1:
        MCI_list.append(result[i])
    else:
        AD_list.append(result[i])

Normal_list = np.array(Normal_list)
MCI_list = np.array(MCI_list)
AD_list = np.array(AD_list)
#print(Normal_list.shape)
#print(MCI_list.shape)
#print(AD_list.shape)
fig = plt.figure(figsize=(8, 8))

plt.scatter(Normal_list[:,0], Normal_list[:, 1], marker='o', c='r')
plt.scatter(MCI_list[:,0], MCI_list[:, 1], marker='s', c='g')
plt.scatter(AD_list[:,0], AD_list[:, 1], marker='^', c='b')

plt.legend(('Normal','MCI','Dementia'))
#plt.title("Thickness",fontdict = {'fontsize' : 30})

plt.show()

In [ ]:
data = np.concatenate((data.array_x,data1.array_x), axis=1)
#print(len(data.array_x))
print(data.shape, data.shape)
print(data.dtype)
#print(data.array_x,data.array_y)

In [ ]:

model_tsne = TSNE(n_components=2,perplexity=50)
#print(i)
result = model_tsne.fit_transform(data)
#print('done')
Normal_list=list()
MCI_list=list()
AD_list=list()
for i in range(result.shape[0]):
    if data1.array_y[i]==0:
        Normal_list.append(result[i])
    elif data1.array_y[i]==1:
        MCI_list.append(result[i])
    else:
        AD_list.append(result[i])

Normal_list = np.array(Normal_list)
MCI_list = np.array(MCI_list)
AD_list = np.array(AD_list)
#print(Normal_list.shape)
#print(MCI_list.shape)
#print(AD_list.shape)
fig = plt.figure(figsize=(8, 8))

plt.scatter(Normal_list[:,0], Normal_list[:, 1], marker='o', c='r')
plt.scatter(MCI_list[:,0], MCI_list[:, 1], marker='s', c='g')
plt.scatter(AD_list[:,0], AD_list[:, 1], marker='^', c='b')

plt.legend(('Normal','MCI','Dementia'))
#plt.title("Thickness",fontdict = {'fontsize' : 30})

plt.show()